In [8]:
import os
import pickle
import random
from datetime import datetime

import numpy as np
import xarray as xr
import plotly.graph_objects as go

from src.field_topology.topology_utils import (
    trace_field_line_rk,
    classify,
)

# ======================================================
# SETTINGS
# ======================================================
case = "CPN"

input_folder = f"/Volumes/data_backup/mercury/extreme/{case}_Base/plane_product/object/"
output_folder = f"/Users/danywaller/Projects/mercury/extreme/bfield_topology/{case}_Base/"
cache_folder = os.path.join(output_folder, "cache")

os.makedirs(output_folder, exist_ok=True)
os.makedirs(cache_folder, exist_ok=True)

# Planet parameters
RM = 2440.0   # Mercury radius [km]

if case in ["RPS", "RPN", "CPS", "CPN"]:
    plot_depth = RM
else:
    raise ValueError("Invalid case")

# Field-line tracing parameters
n_lat = 90
n_lon = n_lat * 2
max_steps = 10000
h_step = 25.0
surface_tol = 75.0

# Plotting
max_lines = 500
colors = {"closed": "blue", "open": "red"}


In [9]:
# ======================================================
# DISCOVER FILES
# ======================================================
ncfiles_all = sorted([
    os.path.join(input_folder, f)
    for f in os.listdir(input_folder)
    if f.startswith(f"Amitis_{case}_Base_") and f.endswith(".nc")
])

if len(ncfiles_all) < 18:
    raise RuntimeError(f"Only found {len(ncfiles_all)} files, need 18")

ncfiles = ncfiles_all[-18:]

print(f"Found {len(ncfiles)} files")

Found 18 files


In [10]:
# ======================================================
# CREATE SEEDS ON SPHERE
# ======================================================
lats = np.linspace(-90, 90, n_lat)
lons = np.linspace(-180, 180, n_lon)

seeds = []
for lat in lats:
    for lon in lons:
        phi = np.radians(lat)
        theta = np.radians(lon)
        seeds.append([
            plot_depth * np.cos(phi) * np.cos(theta),
            plot_depth * np.cos(phi) * np.sin(theta),
            plot_depth * np.sin(phi)
        ])

seeds = np.asarray(seeds)

In [11]:
# ======================================================
# LOAD FIELD
# ======================================================
def load_field(ncfile):
    ds = xr.open_dataset(ncfile)

    x = ds["Nx"].values
    y = ds["Ny"].values
    z = ds["Nz"].values

    Bx = ds["Bx_tot"].isel(time=0).values
    By = ds["By_tot"].isel(time=0).values
    Bz = ds["Bz_tot"].isel(time=0).values

    ds.close()

    # Nz, Ny, Nx → Nx, Ny, Nz
    Bx = np.transpose(Bx, (2, 1, 0))
    By = np.transpose(By, (2, 1, 0))
    Bz = np.transpose(Bz, (2, 1, 0))

    return x, y, z, Bx, By, Bz


In [12]:
# ======================================================
# TRACE + CACHE
# ======================================================
def cache_name(ncfile):
    base = os.path.basename(ncfile).replace(".nc", "")
    return os.path.join(cache_folder, f"{base}_lines.pkl")

lines_by_time = []

for it, ncfile in enumerate(ncfiles):
    cache_file = cache_name(ncfile)

    if os.path.exists(cache_file):
        print(f"[CACHE] Loading {os.path.basename(cache_file)}")
        with open(cache_file, "rb") as f:
            lines_by_topo = pickle.load(f)
        lines_by_time.append(lines_by_topo)
        continue

    print(f"[TRACE] {os.path.basename(ncfile)} ({it+1}/{len(ncfiles)})")
    start = datetime.now()

    x, y, z, Bx, By, Bz = load_field(ncfile)

    lines_by_topo = {"open": []}

    for seed in seeds:
        traj_fwd, exit_fwd_y = trace_field_line_rk(
            seed, Bx, By, Bz, x, y, z,
            plot_depth,
            max_steps=max_steps,
            h=h_step
        )

        traj_bwd, exit_bwd_y = trace_field_line_rk(
            seed, Bx, By, Bz, x, y, z,
            plot_depth,
            max_steps=max_steps,
            h=-h_step
        )

        topo = classify(
            traj_fwd,
            traj_bwd,
            plot_depth + surface_tol,
            exit_fwd_y,
            exit_bwd_y
        )

        if topo == "open":
            lines_by_topo["open"].append(traj_fwd)
            lines_by_topo["open"].append(traj_bwd)

    with open(cache_file, "wb") as f:
        pickle.dump(lines_by_topo, f)

    print(f"  Done in {datetime.now() - start}")
    lines_by_time.append(lines_by_topo)

[CACHE] Loading Amitis_CPN_Base_098000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_099000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_100000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_101000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_102000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_103000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_104000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_105000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_106000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_107000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_108000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_109000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_110000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_111000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_112000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_113000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base_114000_xz_comp_lines.pkl
[CACHE] Loading Amitis_CPN_Base

In [13]:
# ======================================================
# PLANET SPHERE
# ======================================================
theta = np.linspace(0, np.pi, 100)
phi = np.linspace(0, 2*np.pi, 200)
theta, phi = np.meshgrid(theta, phi)

xs = plot_depth * np.sin(theta) * np.cos(phi)
ys = plot_depth * np.sin(theta) * np.sin(phi)
zs = plot_depth * np.cos(theta)

mask_pos = xs >= 0
mask_neg = xs <= 0

planet_traces = [
    go.Surface(
        x=np.where(mask_pos, xs, np.nan),
        y=np.where(mask_pos, ys, np.nan),
        z=np.where(mask_pos, zs, np.nan),
        colorscale=[[0, "lightgrey"], [1, "lightgrey"]],
        showscale=False,
        lighting=dict(ambient=1),
        hoverinfo="skip"
    ),
    go.Surface(
        x=np.where(mask_neg, xs, np.nan),
        y=np.where(mask_neg, ys, np.nan),
        z=np.where(mask_neg, zs, np.nan),
        colorscale=[[0, "black"], [1, "black"]],
        showscale=False,
        lighting=dict(ambient=1),
        hoverinfo="skip"
    )
]

In [22]:
# ======================================================
# BUILD OPEN-LINE FRAMES (CORRECT WAY)
# ======================================================

def get_sim_time(ncfile):
    """
    Extract sim_step from filename and convert to seconds
    """
    base = os.path.basename(ncfile).replace(".nc", "")
    parts = base.split("_")
    try:
        sim_step = int(parts[3])  # third numeric field
        sim_time_s = sim_step * 0.002
        return sim_time_s
    except (IndexError, ValueError):
        return None

frames = []

# Get simulation time of the first file
t0_sec = get_sim_time(ncfiles[0])

camera = dict(
    eye=dict(x=1.5, y=1.5, z=1.5)  # adjust for initial view
)

n_open_max = max(len(t["open"]) for t in lines_by_time)
n_open_plot = min(n_open_max, max_lines)

for it, ncfile in enumerate(ncfiles):
    lines_by_topo = lines_by_time[it]
    traces = []

    open_lines = lines_by_topo.get("open", [])
    if len(open_lines) > n_open_plot:
        open_lines = random.sample(open_lines, n_open_plot)

    for i in range(n_open_plot):
        if i < len(open_lines):
            traj = open_lines[i]
            traces.append(
                go.Scatter3d(
                    x=traj[:,0],
                    y=traj[:,1],
                    z=traj[:,2],
                    mode="lines",
                    line=dict(color="red", width=2),
                    showlegend=False
                )
            )
        else:
            traces.append(
                go.Scatter3d(x=[], y=[], z=[], mode="lines", showlegend=False)
            )

    # Get sim time
    t_sec = get_sim_time(ncfile)

    # Create frame with title
    frames.append(
        go.Frame(
            data=traces,
            name=str(it),
            layout=dict(
                title=dict(
                    text=f"{case.replace('_',' ')} at t = {t_sec:.3f} s",
                    x=0.5, xanchor="center"
                ),
                scene=dict(camera=camera)  # lock camera
            ),
            traces=list(range(len(planet_traces),
                              len(planet_traces)+n_open_plot))
        )
    )


In [25]:
# ======================================================
# FIGURE + SLIDER
# ======================================================
# Initial open lines (t = 0)
init_open = frames[0].data

fig = go.Figure(
    data=planet_traces + list(frames[0].data),
    frames=frames,
    layout=dict(
        title=dict(
            text=f"{case.replace('_',' ')} at t = {t0_sec:.3f} s",
            x=0.5, xanchor="center"
        ),
        scene=dict(camera=camera),
        width=1000,
        height=800
    )
)

fig.update_layout(template="plotly",
    scene=dict(
        xaxis=dict(range=[-3*RM, 3*RM], title="X [km]"),
        yaxis=dict(range=[-3*RM, 3*RM], title="Y [km]"),
        zaxis=dict(range=[-3*RM, 3*RM], title="Z [km]"),
        aspectmode="cube"
    ),

    sliders=[{
        "steps": [
            {
                "method": "animate",
                "args": [[str(i)], {"mode": "immediate", "frame": {"duration":0, "redraw":True}}],
                "label": str(i)
            } for i in range(len(frames))
        ]
    }]
)


# ======================================================
# SAVE
# ======================================================
out_html = "bfield_topology_timeslider.html"
fig.write_html(os.path.join(output_folder, out_html), include_plotlyjs="cdn")
print("Saved:", out_html)


Saved: bfield_topology_timeslider.html
